In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [42]:
using LinearAlgebra

function Make_D1_D2(bl,br,wl,wr,r, D1_til, D2_til)
    D1= [I(bl)        zeros(bl,r) zeros(bl,br);
         zeros(r,bl)  D1_til      zeros(r,br);
         zeros(wl,bl) zeros(wl,r) zeros(wl,br)]

    #D2 in our format
    D2= [zeros(wr,bl) zeros(wr,r) zeros(wr,br);
         zeros(r,bl)  D2_til      zeros(r,br);
         zeros(br,bl) zeros(br,r) I(br)]

    return D1, D2
end

function Make_A_B(bl,br,wl,wr,r,D1_til,D2_til,H,U,V)
    D1,D2 = Make_D1_D2(bl,br,wl,wr,r, D1_til, D2_til)

    A = U * D1 * H
    B = V * D2 * H

    return A, B
end

function Our_SVD(A,B)
    bl,br,wl,wr,r = wire_size(A,B)
    U, V, Q, D1, D2_julia, R0 = svd(A, B)

    P, D2_our = permutation(wr,r,br, D2_julia)
    V_til = V * P #Changing variable to fix line changes in D2

    #Selects D1_tilde and D2_tilde
    D1_til = D1[bl+1:bl+r, bl+1:bl+r]
    D2_til = D2_our[wr+1:wr+r, bl+1:bl+r]

    H = R0 * Q'

    return U, V_til, H, D1_til, D2_til, D1, D2_our, bl, br, wl, wr, r, P, V
end

function wire_size(A,B)
    m,.. = size(A)
    p,.. = size(B)
    .., .., .., D1, D2, .. = svd(A, B)
    ..,k_mais_l = size(D1)

    #Counting k: D1 is a m-by-(k_mais_l) diagonal matrix with 1s in the first k entries,
    k = 0
    for i in 1:min(m, k_mais_l)
        if D1[i, i] == 1
            k += 1
        else
            k += 0
        end
    end

    l = k_mais_l-k

    #Counting r: D2 is a matrix whose upper-right l-by-l block is diagonal,
    #   with the first r entries belonging to D2_til and the rest 1s.
    r = 0
    for i in 1:l
        if D2[i, k+i] != 1
            r += 1
        else
            r += 0
        end
    end

    bl = k
    br = l - r
    wl = m - k - r
    wr = p - l

    return bl,br,wl,wr,r
end

function permutation(wr,r,br, D2_julia)
    P = [zeros(r,wr)  I(r)        zeros(r,br);
         zeros(br,wr) zeros(br,r) I(br);
         I(wr)        zeros(wr,r) zeros(wr,br)]

    P_til = [zeros(wr,r) zeros(wr,br) I(wr);
             I(r)        zeros(r,br)  zeros(r,wr);
             zeros(br,r) I(br)        zeros(br,wr)]

    D2_our = P_til * D2_julia

    return P, D2_our
end

permutation (generic function with 1 method)

In [43]:
function op_switch(D1_til, D2_til, bl, br, wl, wr, r)
    # Cria as novas matrizes de zeros com as dimensões corretas
    D1_new = zeros(bl + r + wl, bl + r + wl)
    D2_new = zeros(wr + r + br, bl + r + wl)

    # Monta o novo D1
    D1_new[1:bl, 1:bl] .= I(bl)                   # bloco identidade inicial
    D1_new[bl+1:bl+r, bl+1:bl+r] .= D1_til         # bloco D1_til no meio
    D1_new[bl+r+1:end, bl+r+1:end] .= I(wl)        # bloco identidade no final

    # Monta o novo D2
    D2_new[wr+1:wr+r, bl+1:bl+r] .= D2_til         # bloco D2_til no meio
    # (resto de D2_new já é zero, não precisa fazer mais nada)

    return D1_new, D2_new
end


op_switch (generic function with 2 methods)

In [44]:
B = [0 1 0
     0 0 1]

A = [1 0 0
     0 1 0]

2×3 Matrix{Int64}:
 1  0  0
 0  1  0

In [45]:
U, V_til, H, D1_til, D2_til, D1, D2_our, bl, br, wl, wr, r, P, V = Our_SVD(A,B)
display(D1)
display(D2_our)
display(bl)
display(wl)
display(r)
display(br)
display(wr)

2×3 Matrix{Float64}:
 1.0  0.0       0.0
 0.0  0.707107  0.0

2×3 Matrix{Float64}:
 0.0  0.707107  0.0
 0.0  0.0       1.0

1

0

1

1

0

In [49]:
D1_new, D2_new =  op_switch(D1_til, D2_til, D1, D2_our, bl, br, wl, wr, r)
#display(D1)
display(D1_new)

#display(D2_our)
display(D2_new)

display(D2_new*inv(D1_new))

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  0.707107

2×2 Matrix{Float64}:
 0.0  0.707107
 0.0  0.0

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  0.0

In [60]:
function aprox(D1_til, D2_til, n)
    # Faz cópias para não alterar as matrizes originais
    D1_til_new = copy(D1_til)
    D2_til_new = copy(D2_til)

    # Altera os primeiros n elementos da diagonal
    for i in 1:n
        D1_til_new[i, i] = 1
        D2_til_new[i, i] = 0
    end

    return D1_til_new, D2_til_new
end

function op_aprox(D1_til, D2_til, bl, br, wl, wr, r, n)
    D1_new = zeros(bl + r + wl, bl + r + wl)
    D2_new = zeros(wr + r + br, bl + r + wl)

    D1_til_aprox, D2_til_aprox = aprox(D1_til, D2_til, n)

    # Monta o novo D1
    D1_new[1:bl, 1:bl] .= I(bl)
    D1_new[bl+1:bl+r, bl+1:bl+r] .= D1_til_aprox

    if wl > 0
        D1_new[bl+r+1:end, bl+r+1:end] .= I(wl)
    end

    # Monta o novo D2
    D2_new[wr+1:wr+r, bl+1:bl+r] .= D2_til_aprox
    # (o resto já é zero)

    return D1_new, D2_new
end



op_aprox (generic function with 2 methods)

In [61]:
D1_aprox, D2_aprox = op_aprox(D1_new, D2_new, bl, br, wl, wr, r, 1)
display(D1_aprox)



LoadError: DimensionMismatch: array could not be broadcast to match destination